<a href="https://colab.research.google.com/github/Jofouda/france-geojson/blob/master/BOT%E2%80%99NORRIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I- Site WEB** : "*Chuck Norris Jokes and Facts*"


**CODE HTML DU SITE**

Clic droit et on peut générer le code source de la page  .
On peut également sélectionner une des information que l’on souhaite récupérer . 


("view-source:https://chucknorrisfacts.net/facts")


In [4]:
import requests
url = 'https://chucknorrisfacts.net/facts'
response = requests.get(url)     
print(response)


<Response [406]>


**II- Conception bot (script python)**
Choix de partie du code html qui nous intéresse.
Utilisation de request et BeautifulSoup

code de récupération des informations 

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import psycopg2, config2, random

# Code SQL pour PostgreSQL


# connection à PG, bdd_jfoudambezele
conn = psycopg2.connect(database="bdd_jfoudambezele", user=config2.user,password=config2.password, host='127.0.0.1') 
cur = conn.cursor()



# Traitement de chaque ligne: affichage & enregistrement
def traiteInfo(id, rate, vote, fact):
    print("%4d : %.2f %5d %s" % (id, rate, vote, fact))
    cur.execute("""INSERT INTO public."BOT_NORRIS" VALUES (%s, %s) ON CONFLICT DO NOTHING;""", (id, fact))
    cur.execute("""INSERT INTO public."BOT_NORRISvote" VALUES (NOW()::Date, %s, %s, %s) ON CONFLICT DO NOTHING;""", (id, rate, vote))

# Definition de la procédure qui traite 1 page
def recupPage(page):
    url = "https://chucknorrisfacts.net/facts.php?page=%d" % (page)
    print("\nRécupération de %s" %(url))
    # extraction du document HTML
    r = requests.get(url, headers={"User-Agent": "Mon navigateur perso d'ici"})
    # Traitement avec la librairie BS
    soup = BeautifulSoup(r.content, 'html.parser')
    # Récupération de tous les blocks qui contiennent les info qui nous intéressent.
    # Utilisation de soup.select avec un selecteur CSS
    blocks = soup.select("#content > div:nth-of-type(n+2)")
    #print(blocks[1])
    # 2ime boucle sur les block récupérée
    for block in blocks: 
        #~ print(block)
        # On récupé les champs individuels (rate, vote, fact)
        # On affiche (si fact non vide)
        fact = block.select_one("p")
        if fact is not None:
            id = block.select_one("ul.star-rating").attrs['id']
            #print(id)
            rate = block.select_one("span.out5Class")
            vote = block.select_one("span.votesClass")
            
            traiteInfo(int(id[6:]), float(rate.text), int(vote.text[:-6]), fact.text)
            #~ print("------------------------")
    


def lastPage():
    url = "https://chucknorrisfacts.net/facts"
    r = requests.get(url, headers={"User-Agent": "Mon navigateur perso d'ici"})
    soup = BeautifulSoup(r.content, 'html.parser')
    lastPage = soup.select("#content a:link")
    lastPageToStr = str(lastPage[-1].get('href'))
    
    numPage = re.findall(r'\d+', lastPageToStr)
    
    return(int(numPage[0]))


for p in range(1, lastPage()):
    recupPage(page = p)
    time.sleep(0.50)   



conn.commit()
conn.close()

**III-Quelques requêtes**

In [ ]:
import psycopg2
from sqlalchemy import create_engine
import config2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


server = "127.0.0.1" 
BDname="bdd_jfoudambezele" # Nom de la base de donnée
alchemyEngine = create_engine('postgresql+psycopg2://' + config2.user + ':' + config2.password + '@' + server + '/' + BDname).connect()

dbConnection = alchemyEngine.connect();
#afficher la table BOT_NORRISvote
sql1 = ("""SELECT * FROM "BOT_NORRISvote" ORDER BY id ASC""")
df1 = pd.read_sql(sql1, dbConnection)
print(df1)


sql16 = ("""SELECT "fact" FROM "BOT_NORRIS" ORDER BY id ASC""")
df16 = pd.read_sql(sql16, dbConnection)
print(df16)






sql5 = (""" SELECT AVG(vote) FROM "BOT_NORRISvote" """)
df5 = pd.read_sql(sql5, dbConnection)
print(df5)


sql6 =(""" SELECT "fact" FROM "BOT_NORRIS" LEFT JOIN "BOT_NORRISvote" ON 'BOT_NORRIS.id'='BOT_NORRISvote.id' """)
df6 = pd.read_sql(sql6, dbConnection)
print(df6)




sql7 =(""" SELECT fact, vote FROM "BOT_NORRIS" LEFT JOIN "BOT_NORRISvote" ON 'BOT_NORRIS.id'='BOT_NORRISvote.id' """)
df7 = pd.read_sql(sql7, dbConnection)
print(df7)

sql8 =(""" SELECT MIN(vote) AS Smallestvote FROM "BOT_NORRISvote" """)
df8 = pd.read_sql(sql8, dbConnection)
print(df8)



sql11 =(""" SELECT MAX(vote) AS Largest FROM "BOT_NORRISvote" """)
df11 = pd.read_sql(sql11, dbConnection)
print(df11)

sql40 = ("""SELECT id, vote FROM "BOT_NORRISvote" WHERE 'vote'<= (SELECT MAX('vote')FROM "BOT_NORRISvote")""")
df40 = pd.read_sql(sql40, dbConnection)
print(df40)
sns.barplot(df40['id'],df40['vote'])



sql12 = (""" SELECT "vote" FROM "BOT_NORRISvote"  WHERE vote=7468""")
df12 = pd.read_sql(sql12, dbConnection)
print(df12)

sql13 = (""" SELECT "vote" FROM "BOT_NORRISvote"  WHERE vote=5""")
df13 = pd.read_sql(sql13, dbConnection)
print(df13)


sql14 = (""" SELECT "vote" FROM "BOT_NORRISvote"  WHERE vote BETWEEN 5 AND 7468 ORDER BY vote ASC""")
df14 = pd.read_sql(sql14, dbConnection)
print(df14)


sql15 = (""" SELECT COUNT(vote) FROM "BOT_NORRISvote"  WHERE vote BETWEEN 5 AND 7468 """)
df15 = pd.read_sql(sql15, dbConnection)
print(df15)


sql9 = (""" SELECT *FROM "BOT_NORRISvote"  WHERE vote= 5  ORDER BY vote ASC """)
df9 = pd.read_sql(sql9, dbConnection)
print(df9)

sql10 = (""" SELECT vote, rate FROM "BOT_NORRISvote" """)
df10 = pd.read_sql(sql10, dbConnection)
print(df10)

sql33 = ("""SELECT   AVG(vote) FROM "BOT_NORRISvote" """)
df33 = pd.read_sql(sql33, dbConnection)
print(df33)

sql34 = """SELECT id, AVG(vote) FROM "BOT_NORRISvote" GROUP BY id """
df34 = pd.read_sql(sql34, dbConnection)
print(df34)



sql26 = ("""SELECT COUNT(fact) FROM "BOT_NORRIS" """ )
df26 = pd.read_sql(sql26, dbConnection)
print(df26)

sql21 = ("""SELECT id, rate FROM "BOT_NORRISvote"  ORDER BY  rate  ASC""")
df21 = pd.read_sql(sql21, dbConnection)
print(df21)



sql28 = ("""SELECT id, fact FROM "BOT_NORRIS" WHERE LENGTH(fact) <50""")
df28 = pd.read_sql(sql28, dbConnection)
print(df28)

sql23 = ("""SELECT COUNT(fact) FROM "BOT_NORRIS" WHERE LENGTH(fact) <50""")
df23 = pd.read_sql(sql23, dbConnection)
print(df23)


sql27 = ("""SELECT id, fact FROM "BOT_NORRIS" WHERE LENGTH(fact) BETWEEN 50 and 200""")
df27 = pd.read_sql(sql27, dbConnection)
print(df27)

sql29 = ("""SELECT COUNT(fact) FROM "BOT_NORRIS" WHERE LENGTH(fact) BETWEEN 50 and 200""")
df29 = pd.read_sql(sql29, dbConnection)
print(df29)

sql24 = ("""SELECT id, fact FROM "BOT_NORRIS" WHERE LENGTH(fact) > 200""")
df24 = pd.read_sql(sql24, dbConnection)
print(df24)


sql35 = (""" SELECT MAX(LENGTH(fact)) FROM "BOT_NORRIS" """)
df35 = pd.read_sql(sql35, dbConnection)
print(df35)

sql36 = (""" SELECT MIN(LENGTH(fact)) FROM "BOT_NORRIS" """)
df36 = pd.read_sql(sql36, dbConnection)
print(df36)

sql37 = (""" SELECT id, LENGTH(fact) FROM "BOT_NORRIS" """)
df37 = pd.read_sql(sql37, dbConnection)
print(df37)




conn.commit()
conn.close()

**IV- Quelques Graphiques**

In [ ]:
import psycopg2
from sqlalchemy import create_engine
import config2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


server = "127.0.0.1" 
BDname="bdd_jfoudambezele" # Nom de la base de donnée
alchemyEngine = create_engine('postgresql+psycopg2://' + config2.user + ':' + config2.password + '@' + server + '/' + BDname).connect()

dbConnection = alchemyEngine.connect();

sns.scatterplot(df37['id'], df37['length'])
sns.barplot(df37['id'], df37['length'])

sns.scatterplot(df34['id'], df34['avg'])
sns.barplot(df34['id'], df34['avg'])

sns.scatterplot(df10['rate'],df10['vote'])
sns.barplot(df10['rate'],df10['vote'])

sns.scatterplot(df11['id'],df11['vote'])
sns.barplot(df11['id'], df11['vote'])


conn.commit()
conn.close()





 Représentations sur les slides pdf.